# Markov Chain Generation
In this notebook, we generate the necessary Markov Chains for each author which will be used as likelihood functions during the identification process.

In [1]:
import string
import sys
import re
from pickle import dump

def pickleDump(filename, todump):
    out = open(filename, 'wb+')
    for d in todump:
        dump(d, out)
    out.close()
    
class MarkovChain():
    """
    Markov Chain class. 
    chain:       nested dictionary representing the number of occurences of a word given the previous word.
    wordCount:   dictionary of the number of total number of words (value) that have occured after the previous word (key).
    """
    
    def __init__(self, author):
        """
        Parameter: 
            - author: name of author (string)
        """
        self.author = author
        self.chain = {'*': {}}
        self.wordCount = {'*': 0}
    
    def addWord(self, prevWord, word):
        """
        Add a word to the Markov Chain.
        Takes the previous word and the current word as strings
        """
        self.chain[prevWord][word] = 1 + self.chain[prevWord].get(word, 0)
        self.wordCount[prevWord]= 1 + self.wordCount.get(prevWord, 0)
    
        # When encountering a new word, add it to the prefix dictionary
        if not self.chain.get(word):
            self.chain[word] = {}
        
    def addSentence(self, sentence):
        """
        Process a "sentence" to produce a Markov Chain. Takes a sentence as a list of lowercase words.
        NO ASTERISKS. SERIOUSLY.
        """
        # '*' represents the beginning of a sentence in the chain. 
        # This way, we can determine the probability of a word starting a sentence
        sentence = ['*'] + sentence
        if len(sentence) > 1:
            for i in [i + 1 for i in range(len(sentence)-1)]: # Start at the second word
                self.addWord(sentence[i-1], sentence[i])
        
    def getProb(self, prevWord, word):
        """
        Return the probability of getting word given prevWord.
        Takes two strings.
        """
        return self.chain[prevWord][word]/self.wordCount[prevWord]
    
def processGutenberg(fileName, author, make=True):
    """
    Process a Gutenberg text file.
    fileName: string
    author: string
    returns a markovChain object.
    """
    f = open(fileName)
    
    #Skip to the beginning of the actual text
    for line in f:
        if line.startswith("*** START OF THIS PROJECT"):
            break
    
    text = ''
    
    # Put the text into one big string
    for line in f:
        # Stop when hitting the end of the book
        if line.startswith("*** END OF THIS PROJECT"):
            break
        
        text += line + ' '
        
    sentences = re.split('[.?!]', text) # Seperate text into a list of sentences sentence
    
    listOSentences = []
    for sentence in sentences:
        # Make all words lowercase and strip off punctuation
        sentenceList = ''.join(char for char in sentence if char in set(string.letters + string.digits + ' ')).lower().split()
        
        if sentenceList != []:
            listOSentences.append(sentenceList)
    
    f.close()
    
    if make==True:
        return makeMarkov(listOSentences, author)
    else:
        return listOSentences

def makeMarkov(sentenceList, author):
    markovChain = MarkovChain(author)
    for sentence in sentenceList:
        # Process the sentence in the Markov Chain
        markovChain.addSentence(sentence)
    return markovChain

In [50]:
greatExp = processGutenberg('GreatExpectations.txt', 'Charles Dickens')
frank = processGutenberg('Frankenstein.txt', 'Mary Shelley')
romeoJuliet = processGutenberg('RomeoAndJuliet.txt', 'Shakespeare')

pickleDump('GreatExp.dat', [greatExp.chain, greatExp.wordCount, greatExp.author])
pickleDump('Frankenstein.dat', [frank.chain, frank.wordCount, frank.author])
pickleDump('RomeoJuliet.dat', [romeoJuliet.chain, romeoJuliet.wordCount, romeoJuliet.author])

In [68]:
# Update test Text
unknown = processGutenberg('testText.txt', 'Unknown', make=False)
pickleDump('testText.dat', [unknown])

In [46]:
greatExp.wordCount
count = 0
for key, val in romeoJuliet.wordCount.iteritems():
    if val > 10000:
        print(key, val)
        count += 1
print(greatExp.wordCount['*'])

('the', 16187)
('and', 12056)
('*', 25974)
25974
